In [7]:
import json
import pandas as pd
import itertools
import numpy as np
from collections import Counter
import re
import csv
from nltk.corpus import stopwords

In [8]:
def show_K_df(file):
    with open(file) as json_file:
        data = json.load(json_file)
    s1 = []
    for i in range(len(data)):
        s1.append(list('s1_'+ str(i) for i in list(data[i].values())[0]['s1']))
        s1_list = [item for sublist in s1 for item in sublist]
    s2 = []
    for i in range(len(data)):
        s2.append(list('s2_'+ str(i) for i in list(data[i].values())[0]['s2']))
        s2_list = [item for sublist in s2 for item in sublist]
    k = sum(list(Counter(s1_list).values()))+ sum(list(Counter(s2_list).values())) / 300
    k = int(k)-1
    
    df = pd.DataFrame([s1,s2]).T
    
    return df, k
    

In [9]:
df, k = show_K_df('blocks.json')

In [10]:
def each_comparison(df):

    comparison = []
    for m in range(len(df)):
        for i in df[0][m]:
            for j in df[1][m]:
                comparison.append(i + " // "+ j)
    return comparison

In [11]:
comparison = each_comparison(df)

In [12]:
#Initicial Number of Comparsion
len(comparison)

21254

In [13]:
#the number of blocks two entities have in common
def common_block_scheme(comparison):
    count = Counter(comparison)
    return count

In [14]:
count = common_block_scheme(comparison)

In [15]:
#Before Pruning
#count

In [16]:
def cardinality_node_pruning(count,k):
    
    countdf = pd.DataFrame.from_dict(count, orient='index').reset_index()
    countdf = countdf.sort_values(by=[0], ascending=False)
    arr = np.array(countdf[0])
    length = len(arr[arr.cumsum() <= k])
    output  = countdf[:length]
    #output = output.rename({'index':0, 0:1})
    pairs = output['index'].str.split('//', expand = True)
    pairs['count'] = output[0]
    
    return pairs    

In [17]:
after_pruning = cardinality_node_pruning(count,k)
#After Pruning
len(after_pruning)

172

# Evaluation 

In [18]:
def extracted_collections(file1, file2, after_pruning):
    first= pd.read_csv(file1, delimiter=';')
    second= pd.read_csv(file2, delimiter=';')
    
    first_id = after_pruning[0].str.replace("s1_","").astype(int)
    second_id = after_pruning[1].str.replace("s2_","").astype(int)
    
    first_extracted = first[first['id'].isin(first_id)]
    second_extracted = second[second['id'].isin(second_id)]
    
    ids = pd.DataFrame([list(first_id),list(second_id)]).T
    return ids, first_extracted, second_extracted

In [19]:
ids, first_extracted, second_extracted = extracted_collections('first.csv', 'second.csv', after_pruning)

In [20]:
def token_description(df,index,group):
    
    row = df[df['id'] == int(ids.iloc[index][group])].values.tolist()
    row = row.pop(0)
    row.pop(0)
    
    res = [sub.split() for sub in row]
    res = [item for sublist in res for item in sublist]
    res = [w for w in res if w !='NONE']
    filtered_word_list = res[:]
    for word in res: 
        if word in stopwords.words('english'): 
            filtered_word_list.remove(word)
    return filtered_word_list
    

In [21]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [28]:
def scroing(ids, first_extracted, second_extracted):
    score =[]
    for i in range(len(ids)):
        list1= token_description(first_extracted,i,0)
        list2= token_description(second_extracted,i,1)
        score.append(jaccard_similarity(list1, list2))
        
    ids['score'] = score
    pred = []
    for i in range(len(ids)):
        if ids['score'][i]>=0.3:
            pred.append(1)
        else:  pred.append(0)
    real=[]
    for i in range(len(ids)):
        if ids[0][i]==ids[1][i]:
            real.append(1)
        else: real.append(0)
    ids['predict'] = pred
    ids['real'] = real
   
    return ids       

In [29]:
result = scroing(ids, first_extracted, second_extracted)

In [30]:
result.head(5)

,0,1,score,predict,real
0,2948,2948,0.658537,1,1
1,888,888,0.923077,1,1
2,1108,1108,0.758621,1,1
3,863,863,0.894737,1,1
4,827,827,0.772727,1,1


In [31]:
print("Pairs Quality(PQ): ",sum(result.predict)/len(after_pruning))

Pairs Quality(PQ):  0.46511627906976744


In [32]:
print("Pairs Completeness(PC): ",sum(result.real)/90)

Pairs Completeness(PC):  0.8222222222222222
